In [29]:
# Let's load on dependencies for this notebook
import argparse
import isce
import isceobj
import os
from osgeo import gdal
from osgeo import gdal_array
import numpy as np
import xml.etree.ElementTree as ET


In [30]:
###Folowing files need to be the same size as your output grid.
latfile  = '/scratch/alor_2015_dsc_III/geometry/lat.rdr.full.vrt'
lonfile  = '/scratch/alor_2015_dsc_III/geometry/lon.rdr.full.vrt'

In [31]:
###Files to resample from geo coords
#infile       = 'geo.int'
infile       = '/scratch/alor_2015_dsc_III/interferogram/filt_topophase.flat'
resampMethod = 'near'
outfile      = 'ungeo.int'

In [32]:
def writeVRT(infile, latFile, lonFile):
#This function is modified from isce2gis.py
            tree = ET.parse(infile + '.vrt')
            root = tree.getroot()

            meta = ET.SubElement(root, 'metadata')
            meta.attrib['domain'] = "GEOLOCATION"
            meta.tail = '\n'
            meta.text = '\n    '


            rdict = { 'Y_DATASET' : latFile,
                      'X_DATASET' : lonFile,
                      'X_BAND' : "1",
                      'Y_BAND' : "1",
                      'PIXEL_OFFSET': "0",
                      'LINE_OFFSET' : "0",
                      'LINE_STEP' : "1",
                      'PIXEL_STEP' : "1" }

            for key, val in rdict.items():
                data = ET.SubElement(meta, 'mdi')
                data.text = val
                data.attrib['key'] = key
                data.tail = '\n    '

            data.tail = '\n'
            tree.write(infile + '.vrt')

In [33]:
##get number of rows and columns in radar coords file (lat)
raster = gdal.Open(latfile)
radarnx=raster.RasterXSize
radarny=raster.RasterYSize
cols,rows=np.meshgrid(np.arange(1,radarnx+1),np.arange(radarny,0,-1))

#make new files of columns and rows, in radar coordinates
colfile = 'cols.r4'
rowfile = 'rows.r4'
gcolfile = 'geo_cols.r4'
growfile = 'geo_rows.r4'
driver=gdal.GetDriverByName('ISCE')
colds = driver.Create(colfile,radarnx,radarny,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(cols)
colds=None

colds = driver.Create(rowfile,radarnx,radarny,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(rows)
colds=None

cmd = 'fixImageXml.py -i '+colfile+' -f'
os.system(cmd)
cmd = 'fixImageXml.py -i '+rowfile+' -f'
os.system(cmd)
writeVRT(colfile, latfile, lonfile)
writeVRT(rowfile, latfile, lonfile)

ERROR 1: Image file is too small


AttributeError: 'NoneType' object has no attribute 'RasterXSize'

In [7]:
##get number of rows and columns + min/max range of input file
raster = gdal.Open(infile)
geonx=raster.RasterXSize
geony=raster.RasterYSize

minLon = raster.GetGeoTransform()[0]
deltaLon = raster.GetGeoTransform()[1]
maxLat = raster.GetGeoTransform()[3]
deltaLat = raster.GetGeoTransform()[5]
minLat = maxLat + geony*deltaLat
maxLon = minLon + geonx*deltaLon
WSEN = str(minLon)+' '+str(minLat)+' '+str(maxLon)+' '+str(maxLat)


In [8]:
#geocode row and col file
cmd = 'gdalwarp -of ISCE -geoloc  -te '+WSEN+' -tr '+str(deltaLon)+' '+str(deltaLat)+' -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear ' +rowfile +'.vrt ' + growfile
print(cmd)
os.system(cmd)
cmd = 'gdalwarp -of ISCE -geoloc  -te '+WSEN+' -tr '+str(deltaLon)+' '+str(deltaLat)+' -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear ' +colfile +'.vrt ' + gcolfile
print(cmd)
os.system(cmd)
cmd = 'fixImageXml.py -i '+gcolfile+' -f'
os.system(cmd)
cmd = 'fixImageXml.py -i '+growfile+' -f'
os.system(cmd)

gdalwarp -of ISCE -geoloc  -te 124.48944444444444 -8.749166666666666 125.26 -7.998055555555555 -tr 0.0002777777777777778 -0.0002777777777777778 -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear rows.r4.vrt geo_rows.r4
gdalwarp -of ISCE -geoloc  -te 124.48944444444444 -8.749166666666666 125.26 -7.998055555555555 -tr 0.0002777777777777778 -0.0002777777777777778 -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear cols.r4.vrt geo_cols.r4


ERROR 1: Output dataset geo_rows.r4 exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset geo_cols.r4 exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /home/karlee/earthquakes/alor_2015_JupNot/dsc_half_fringe_digitized/ungeo2/geo_cols.r4
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /home/karlee/ea

0

In [9]:
###set any geocoding info
WSEN = '0.5 0.5 '+str(radarnx+0.5)+' '+str(radarny+0.5)
writeVRT(infile, growfile, gcolfile)
print('this will just append geoloc info to the vrt - perhaps check to make sure multiple copies are not ending up there')

this will just append geoloc info to the vrt - perhaps check to make sure multiple copies are not ending up there


In [11]:
cmd = 'gdalwarp -of ISCE -geoloc  -te '+ WSEN + ' -tr 1 1 -srcnodata 0 -dstnodata 0 ' + ' -wt cfloat32 -r ' + resampMethod + ' ' + infile +'.vrt ' 'ungeo.int'
print (cmd)
os.system(cmd)
cmd = 'fixImageXml.py -i '+outfile+' -f'
os.system(cmd)

gdalwarp -of ISCE -geoloc  -te 0.5 0.5 8710.5 21348.5 -tr 1 1 -srcnodata 0 -dstnodata 0  -wt cfloat32 -r near /home/karlee/earthquakes/alor_2015_JupNot/dsc_half_fringe_digitized/okapy/model.geo.vrt ungeo.int
Creating output file that is 8710P x 21348L.
Processing /home/karlee/earthquakes/alor_2015_JupNot/dsc_half_fringe_digitized/okapy/model.geo.vrt [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /home/karlee/earthquakes/alor_2015_JupNot/dsc_half_fringe_digitized/ungeo2/ungeo.int


0

In [12]:
# lets resample the ungeocoded model with looks.py 
# if you file doesn't look the same dimensions as original

!looks.py -i ungeo.int -o model_ungeo.int -r 3 -a 8 

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Output filename : model_ungeo.int
GDAL open (R): ungeo.int.vrt
API open (WR): model_ungeo.int
GDAL close: ungeo.int.vrt
API close:  model_ungeo.int
API open (R): model_ungeo.int
API close:  model_ungeo.int
Writing geotrans to VRT for model_ungeo.int


In [ ]:
# at the end of this notebook you will have a ungeocoded model with the corrected x and y values 
# it is called <ungeo_model.int>